# A Reduction Tale

> Objectives:
> * Compare operations taking place in different data containers
> * Compare sizes for these data containers
> * Help deciding when it is best to use a container or another

Let's suppose that we are going to need reductions a lot and we want to choose the best container for performing them.  First, let's start by activating our MemWatcher agent:

In [1]:
if 'MemWatcher' not in dir():
    from ipython_memwatcher import MemWatcher
    mw = MemWatcher()
    mw.start_watching_memory()

In [1] used 0.000 MiB RAM in 0.001s, peaked 0.000 MiB above current, total RAM usage 41.301 MiB


and choose a different container for the data that we want to reduce, starting with a list:

## Regular lists

In [2]:
a = [float(i) for i in range(10*1000*1000)]

In [2] used 311.777 MiB RAM in 1.476s, peaked 0.000 MiB above current, total RAM usage 353.078 MiB


Now, proceed with a simple reduction (sum):

In [3]:
t = %timeit -o sum(a)

10 loops, best of 3: 43.1 ms per loop
In [3] used 0.176 MiB RAM in 1.802s, peaked 0.000 MiB above current, total RAM usage 353.254 MiB


which, in MFLOPS (Mega-FloatingPointOps-Per-Second) is:

In [4]:
print("MFLOPS:", round((len(a) / t.best / 1e6), 1))

MFLOPS: 232.2
In [4] used 0.082 MiB RAM in 0.001s, peaked 0.000 MiB above current, total RAM usage 353.336 MiB


Ok, so that seems fast, but we don't have other references to compare with.  In addition, a list is not the best kind of container in terms of space consumption.  So let's try now a container that seems quite optimal in terms of memory savings.

## NumPy

In [5]:
import numpy as np

In [5] used 13.430 MiB RAM in 0.235s, peaked 0.000 MiB above current, total RAM usage 366.766 MiB


In [6]:
na = np.array(a, dtype=np.float64)

In [6] used 74.504 MiB RAM in 0.199s, peaked 0.000 MiB above current, total RAM usage 441.270 MiB


In [7]:
print("SIZE:", round((na.size * na.itemsize) / 2**20., 3))

SIZE: 76.294
In [7] used 0.008 MiB RAM in 0.001s, peaked 0.000 MiB above current, total RAM usage 441.277 MiB


We see that, with 8 bytes/element, NumPy is a very efficient container.

In [8]:
t = %timeit -o sum(na)

1 loop, best of 3: 712 ms per loop
In [8] used 0.000 MiB RAM in 2.855s, peaked 0.000 MiB above current, total RAM usage 441.277 MiB


In [9]:
print("MFLOPS:", round(len(a) / t.best / 1e6, 3))

MFLOPS: 14.049
In [9] used 0.000 MiB RAM in 0.001s, peaked 0.000 MiB above current, total RAM usage 441.277 MiB


### Exercise

The performance for NumPy is several times slower than the computation with the list.  Why so?

*Hint: * We are using sum() which is a Python function.

### Solution

NumPy has a lot of overhead in producing a Python integer for every element in the array for feeding it to the sum().

*Hint:* Use internal NumPy methods (ufuncs) when possible.

In [10]:
t = %timeit -o na.sum()

100 loops, best of 3: 3.46 ms per loop
In [10] used 0.000 MiB RAM in 1.436s, peaked 0.000 MiB above current, total RAM usage 441.277 MiB


In [11]:
print("FLOPS:", round(len(a) / t.best / 1e6, 3))

FLOPS: 2886.848
In [11] used 0.000 MiB RAM in 0.001s, peaked 0.000 MiB above current, total RAM usage 441.277 MiB


This is more than 100x the speed of sum() on a Python list and it is also pretty optimal in terms of both execution time and space consumed.

## Exercise

The speed in the above reduction is limited by memory speed, not CPU speed.  Could you provide a hint on the maximum memory speed that supports your laptop?

#### Solution

In [12]:
# This is an easy one.  Just divide the size of the dataset by the time that takes the reduction
(na.size * na.itemsize) / t.best / 2**30

21.508695521565222

In [12] used 0.000 MiB RAM in 0.006s, peaked 0.000 MiB above current, total RAM usage 441.277 MiB


So, in this case the memory bandwidth is ~ 17 GB/s.

## Using compressed in-memory containers with bcolz

But let us suppose that we have really big data to process in our laptop and want to see if we can store our data in less space.  Enter compression:

In [13]:
import bcolz
bcolz.print_versions()
bcolz.defaults.cparams['cname'] = 'lz4hc'
bcolz.defaults.cparams['clevel'] = 3
bcolz.defaults.cparams['shuffle'] = bcolz.SHUFFLE
bcolz.set_nthreads(4)

-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
bcolz version:     1.1.1.dev16
bcolz git info:    b'1.1.0-16-g6a40395'
NumPy version:     1.11.1
Blosc version:     1.9.3 ($Date:: 2016-07-06 #$)
Blosc compressors: ['blosclz', 'lz4', 'lz4hc', 'snappy', 'zlib']
Numexpr version:   2.6.1
Dask version:      0.11.0
Python version:    3.5.2 |Continuum Analytics, Inc.| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
Platform:          linux-x86_64
Byte-ordering:     little
Detected cores:    8
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=


8

In [13] used 37.254 MiB RAM in 0.445s, peaked 0.000 MiB above current, total RAM usage 478.531 MiB


In [14]:
ca = bcolz.carray(na)

In [14] used 2.504 MiB RAM in 0.034s, peaked 0.000 MiB above current, total RAM usage 481.035 MiB


In [15]:
print("mem_used:", mw.measurements.memory_delta)

mem_used: 2.50390625
In [15] used 0.000 MiB RAM in 0.001s, peaked 0.000 MiB above current, total RAM usage 481.035 MiB


Also, bcolz containers can provide an estimation on how much memory they are taking; let's have a look:

In [16]:
ca

carray((10000000,), float64)
  nbytes := 76.29 MB; cbytes := 1.40 MB; ratio: 54.51
  cparams := cparams(clevel=3, shuffle=1, cname='lz4hc', quantize=0)
  chunklen := 65536; chunksize: 524288; blocksize: 131072
[  0.00000000e+00   1.00000000e+00   2.00000000e+00 ...,   9.99999700e+06
   9.99999800e+06   9.99999900e+06]

In [16] used 0.539 MiB RAM in 0.002s, peaked 0.000 MiB above current, total RAM usage 481.574 MiB


In this case we see that bcolz estimation is reasonably close to `ipython_memwatcher` measurements.  Let's have a look at the speed of the reduction:

In [17]:
t = %timeit -o ca.sum()
print("MFLOPS:", round(len(a) / t.best / 1e6, 3))

100 loops, best of 3: 12 ms per loop
MFLOPS: 834.72
In [17] used 0.121 MiB RAM in 4.913s, peaked 0.000 MiB above current, total RAM usage 481.695 MiB


This is around 2~5x slower (depending on the machine) than a regular NumPy array, but the size of the array is a quite impressive 50x smaller.  But is compression the only responsible of the overhead?  Let's investigate a bit further.

## Using uncompressed containers with bcolz

In order to see if this is because of the compression overhead, let's use an uncompressed array:

In [18]:
cau = bcolz.carray(a, cparams=bcolz.cparams(clevel=0))

In [18] used 76.535 MiB RAM in 0.259s, peaked 3.242 MiB above current, total RAM usage 558.230 MiB


In [19]:
cau

carray((10000000,), float64)
  nbytes := 76.29 MB; cbytes := 76.50 MB; ratio: 1.00
  cparams := cparams(clevel=0, shuffle=1, cname='lz4hc', quantize=0)
  chunklen := 65536; chunksize: 524288; blocksize: 65536
[  0.00000000e+00   1.00000000e+00   2.00000000e+00 ...,   9.99999700e+06
   9.99999800e+06   9.99999900e+06]

In [19] used 0.000 MiB RAM in 0.002s, peaked 0.000 MiB above current, total RAM usage 558.230 MiB


In [20]:
t = %timeit -o cau.sum()
print("MFLOPS:", round(len(a) / t.best / 1e6, 3))

100 loops, best of 3: 7.42 ms per loop
MFLOPS: 1348.591
In [20] used 0.020 MiB RAM in 3.066s, peaked 0.000 MiB above current, total RAM usage 558.250 MiB


As we can see, the times with an uncompressed `carray` are noticieably faster than with a compressed one, so compressing is not the only source of the overhead.  The other source of the difference is the memory layout of the different containers (bcolz's carray data container layout is a bit more complex than NumPy).

So, while bcolz allows to use compressed in-memory data containers, this usually represents more cost in performance (compared with NumPy).  But sometimes you may prefer to keep more data in-memory and assume that the computations are going to be slower.

## Exercise

bcolz uses Blosc, a multithreaded meta-compressor, to do the compression under the hood.  Blosc can use different codecs, and each one has different behavior in terms of performance.  Given the next computation:

In [21]:
bcolz.defaults.cparams['cname'] = 'blosclz'   # try also with 'lz4', 'lz4hc', 'zlib'
bcolz.defaults.cparams['clevel'] = 9          # try from 1 to 9
bcolz.defaults.cparams['shuffle'] = bcolz.SHUFFLE   # try with bcolz.NOSHUFFLE and bcolz.BITSHUFFLE too
bcolz.set_nthreads(8)
ca = bcolz.carray(na)
%timeit ca.sum()
print("carray sizes--> uncompr: %.3f MB, compr: %.3f MB, ratio: %.3f" % (
    ca.nbytes / 2.**20, ca.cbytes/ 2**20., (ca.nbytes / ca.cbytes)))

100 loops, best of 3: 13.1 ms per loop
carray sizes--> uncompr: 76.294 MB, compr: 0.994 MB, ratio: 76.719
In [21] used 2.859 MiB RAM in 5.397s, peaked 0.000 MiB above current, total RAM usage 561.109 MiB


Play with the different parameters and see:

1) Which provides the best compression

2) Which the fastest speed

3) The combination that strikes a good balance between compression and performance

### Solution

Athough this is somewhat system dependent, the codecs that usually provides the best compression are 'zlib' and 'lz4hc', whereas the ones that provided best speed are 'blosclz' and 'lz4'.  The one that strikes a good trade-off is probably 'lz4' with compression level > 6.